<a href="https://colab.research.google.com/github/compomics/workshop-ml-proteomics/blob/main/1.1%20MS%C2%B2PIP%20spectrum%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>